# 텐서플로우 기초(1)

# 텐서플로우 : 텐서를 흘려보내면서 머신러닝과 딥러닝 알고리즘을 수행하는 라이브러리
- 숫자 1 (스칼라 or rank0 텐서)
- 1차원 배열 (벡터 or rank1 텐서)
- 2차원 배열 (행렬 or rank2 텐서)
- 3차원 배열 (텐서 or rank3 텐서)

텐서들은 그래프 구조에서 노드에서 노드로 흘러감
세션을 만들고 세션을 통해 노드간의 데이터(텐서) 연산 수행

In [2]:
import tensorflow as tf

# 상수 노드 정의
a = tf.constant(1.0, name = 'a') 
b = tf.constant(2.0, name = 'b')
c = tf.constant([ [1.0, 2.0], [3.0, 4.0] ])

# 노드의 상태 출력
print(a); print(a+b); print(c)

# 세션을 만들고 노드간의 텐서 연산 실행
sess = tf.Session()

print(sess.run([a, b]))
print(sess.run(c))
print(sess.run([a+b]))
print(sess.run(c+1.0)) # broadcast 수행

# 세션 close
sess.close()

Tensor("a_1:0", shape=(), dtype=float32)
Tensor("add_3:0", shape=(), dtype=float32)
Tensor("Const_1:0", shape=(2, 2), dtype=float32)
[1.0, 2.0]
[[1. 2.]
 [3. 4.]]
[3.0]
[[2. 3.]
 [4. 5.]]


In [3]:
# 플레이스홀더 노드 정의 (임의의 값을 입력받기 위해)
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
c = a+b

# 세션을 만들고 플레이스홀더 노드를 통해 값 입력받음
sess = tf.Session()

print(sess.run(c, feed_dict = {a:1.0, b:3.0}))
print(sess.run(c, feed_dict = {a:[1.0, 2.0], b:[3.0, 4.0]}))

# 연산추가
d = 100 * c

print(sess.run(d, feed_dict = {a:1.0, b:3.0}))
print(sess.run(d, feed_dict = {a:[1.0, 2.0], b:[3.0, 4.0]}))

sess.close()

4.0
[4. 6.]
400.0
[400. 600.]


# placeholder 노드는 머시러닝/딥러닝에서 입력데이터, 정답데이터를 넣어주기 위한 용도로 주로 사용됨

In [4]:
# 가중치나 바이어스처럼 값이 계속 업데이트되는 변수노드 정의
W1 = tf.Variable(tf.random_normal([1])) # np.random.rand(1) 비슷함
b1 = tf.Variable(tf.random_normal([1]))

W2 = tf.Variable(tf.random_normal([1,2]))
b2 = tf.Variable(tf.random_normal([1,2]))

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for step in range(3) :
    W1 = W1 - step   # W1 변수노드 업데이트
    b1 = b1 - step
    
    W2 = W2- step
    b2 = b2 - step
    
    print("step = ", step, ", W1 = ", sess.run(W1), ", b1 = ", sess.run(b1))
    print("step = ", step, ", W2 = ", sess.run(W2), ", b2 = ", sess.run(b2))

sess.close()

step =  0 , W1 =  [-0.14917932] , b1 =  [2.730621]
step =  0 , W2 =  [[0.0660696 1.835704 ]] , b2 =  [[-0.05446811  1.1360923 ]]
step =  1 , W1 =  [-1.1491793] , b1 =  [1.7306211]
step =  1 , W2 =  [[-0.9339304   0.83570397]] , b2 =  [[-1.0544682  0.1360923]]
step =  2 , W1 =  [-3.1491795] , b1 =  [-0.2693789]
step =  2 , W2 =  [[-2.9339304 -1.164296 ]] , b2 =  [[-3.0544682 -1.8639077]]


# 텐서플로우 기초(2)

# Linear Regression (multi-variable example)
- 입력과 정답데이터로 분리
- y = X*W +b = > 손실함수(MSE)가 최소값인가?
- 아니라면 W,b를 최적화 하기위해 GDA(경사하강법-미분)을 이용하여 반복

In [6]:
import tensorflow as tf
import numpy as np

loaded_data = np.loadtxt('./data-01.csv', delimiter=',') # 25x4

x_data = loaded_data[:, 0:-1] # 25X3
t_data = loaded_data[:, [-1]] # 25X1

print("x_data.shape = ", x_data.shape)
print("t_data.shape = ", t_data.shape)

x_data.shape =  (25, 3)
t_data.shape =  (25, 1)


In [7]:
W = tf.Variable(tf.random_normal([3, 1])) # 3X1
b = tf.Variable(tf.random_normal([1]))

X = tf.placeholder(tf.float32, [None, 3]) # None대신 25써도 됨, None이라 쓴 이유는 차후에 확장하기 위해서
T = tf.placeholder(tf.float32, [None, 1])

In [8]:
y = tf.matmul(X, W) # 현재 X, W, b를 바탕으로 계산된 값
loss = tf.reduce_mean(tf.square(y-T)) # MSE 손실함수 정의

In [9]:
learning_rate = 1e-5

optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [11]:
with tf.Session() as sess :
    sess.run(tf.global_variables_initializer())
    for step in range(8001) :
        loss_val, y_val, _ = sess.run([loss, y, train], feed_dict = {X:x_data, T:t_data}) # feed_dict를 통해 입력데이터와 정답데이터 입력
        # _ : 저장할 이유는 없지만 로테이션을 맞추기 위해
        if step % 400 == 0 :
            print("step = ", step, ", loss_val = ", loss_val)
    
    print("\nPrediction is ", sess.run(y, feed_dict={X:[[100, 98, 81]]}))

step =  0 , loss_val =  90396.66
step =  400 , loss_val =  7.210976
step =  800 , loss_val =  6.9096007
step =  1200 , loss_val =  6.692799
step =  1600 , loss_val =  6.5362945
step =  2000 , loss_val =  6.422879
step =  2400 , loss_val =  6.3403063
step =  2800 , loss_val =  6.279939
step =  3200 , loss_val =  6.2355404
step =  3600 , loss_val =  6.2027583
step =  4000 , loss_val =  6.1783843
step =  4400 , loss_val =  6.1601744
step =  4800 , loss_val =  6.1464806
step =  5200 , loss_val =  6.1361384
step =  5600 , loss_val =  6.1282454
step =  6000 , loss_val =  6.122215
step =  6400 , loss_val =  6.117565
step =  6800 , loss_val =  6.113972
step =  7200 , loss_val =  6.1111627
step =  7600 , loss_val =  6.108971
step =  8000 , loss_val =  6.1072392

Prediction is  [[179.15967]]


# 텐서플로우 기초(3)

# Logistic Regression 
- 입력, 정답데이터 분리
- z = X*W+b => sigmoid(z) = y
- 손실함수가 최소인가
- 아니라면 GDA를 통해 W,b를 업데이트

In [12]:
import tensorflow as tf
import numpy as np

loaded_data = np.loadtxt('./diabetes.csv', delimiter = ',')  # 759x9

x_data = loaded_data[:, 0:-1]
t_data = loaded_data[:,[-1]]

print("loaded_data = ", loaded_data.shape)
print("x_data = ", x_data.shape, "\nt_data = ", t_data.shape)

loaded_data =  (759, 9)
x_data =  (759, 8) 
t_data =  (759, 1)


In [13]:
X = tf.placeholder(tf.float32, [None, 8])
T = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.random_normal([8, 1]))
b = tf.Variable(tf.random_normal([1]))

In [14]:
z = tf.matmul(X, W) + b # 선형회귀 값 z

y = tf.sigmoid(z)       # 시그모이도로 계산값

# 손실함수는 Cross-Entropy
loss = -tf.reduce_mean(T*tf.log(y + (1-T)*tf.log(1-y)))

In [16]:
learning_rate = 0.01

optimizer = tf.train.GradientDescentOptimizer(learning_rate)

train = optimizer.minimize(loss)

In [18]:
predicted = tf.cast(y>0.5, dtype = tf.float32) # 759개 데이터와 비교 y>0.5이면 True 아니면 False
                                               # tf.cast를 통해 True는 1 False는 0으로 리턴
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, T), dtype = tf.float32)) # predicted와 T가 같으면 True 아니면 False 

In [19]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer()) 
    for step in range(20001) :
        loss_val, _ = sess.run([loss, train], feed_dict = {X:x_data, T:t_data})
        if step % 500 == 0:
            print("step = ", step, ", loss_val = ", loss_val)
            
    # Accuracy 확인
    y_val, predicted_val, accuracy_val = sess.run([y, predicted, accuracy], feed_dict = {X:x_data, T:t_data})
    
    print("\ny_val.shape =", y_val.shape, ", predicted_val", predicted_val.shape)
    print("\nAccuracy = ", accuracy_val) 
    # 하이퍼 파라미터 조정, 가중치 초기화 방법 변경 또는 아키텍쳐 변경 등을 통해 정확도 높일 수 있음

step =  0 , loss_val =  nan
step =  500 , loss_val =  nan
step =  1000 , loss_val =  nan
step =  1500 , loss_val =  nan
step =  2000 , loss_val =  nan
step =  2500 , loss_val =  nan
step =  3000 , loss_val =  nan
step =  3500 , loss_val =  nan
step =  4000 , loss_val =  nan
step =  4500 , loss_val =  nan
step =  5000 , loss_val =  nan
step =  5500 , loss_val =  nan
step =  6000 , loss_val =  nan
step =  6500 , loss_val =  nan
step =  7000 , loss_val =  nan
step =  7500 , loss_val =  nan
step =  8000 , loss_val =  nan
step =  8500 , loss_val =  nan
step =  9000 , loss_val =  nan
step =  9500 , loss_val =  nan
step =  10000 , loss_val =  nan
step =  10500 , loss_val =  nan
step =  11000 , loss_val =  nan
step =  11500 , loss_val =  nan
step =  12000 , loss_val =  nan
step =  12500 , loss_val =  nan
step =  13000 , loss_val =  nan
step =  13500 , loss_val =  nan
step =  14000 , loss_val =  nan
step =  14500 , loss_val =  nan
step =  15000 , loss_val =  nan
step =  15500 , loss_val =  nan


# 텐서플로우 기초(4)

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
print("\n", mnist.train.num_examples, mnist.test.num_examples, mnist.validation.num_examples)

print("\ntrain image shape = ", np.shape(mnist.train.images)) # 55000, 784(784 픽셀을 가진 이미지 55000개)
print("train label shape = ", np.shape(mnist.train.labels))   # 55000, 10 (10개 원핫인코딩을 가진 55000개)
print("test image shape = ", np.shape(mnist.test.images))
print("test label shape = ", np.shape(mnist.test.labels))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz

 55000 10000 5000

train image shape =  (55000, 784)
train label shape =  (55000, 10)
test image shape =  (10000, 784)
test label shape =  (10000, 10)


In [7]:
learning_rate = 0.1
epochs = 100
batch_size = 100   # 한번에 입력으로 주어지는 MNIST 개수

input_nodes = 784
hidden_nodes = 100 # 임의로 설정
output_nodes = 10

In [8]:
X = tf.placeholder(tf.float32, [None, input_nodes])
T = tf.placeholder(tf.float32, [None, output_nodes])

W2 = tf.Variable(tf.random_normal([input_nodes, hidden_nodes])) # 은닉층 가중치 노드
b2 = tf.Variable(tf.random_normal([hidden_nodes]))              # 은닉층 바이어스 노드

W3 = tf.Variable(tf.random_normal([hidden_nodes, output_nodes]))# 출력층 가중치 노드
b3 = tf.Variable(tf.random_normal([output_nodes]))              # 출력층 바이어스 노드

In [9]:
Z2 = tf.matmul(X, W2) + b2  # 선형회귀 선형회귀 값 Z2
A2 = tf.nn.relu(Z2) # 은닉층 출력 값 A2, sigmoid 대신 사용

# 출력층 선형회귀 값 Z3, 즉 softmax에 들어가는 입력값
Z3 = logits = tf.matmul(A2, W3) + b3

y = A3 = tf.nn.softmax(Z3)

In [10]:
# 출력층 선형회귀 값(logits) Z3와 정답 T를 이용하여 손실함수 크로스 엔트로피 계산
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z3, labels=T))
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [11]:
# batch_size X 10 데이터에 대한 argmax를 통해 행단위로 비교함
predicted_val = tf.equal(tf.argmax(A3, 1), tf.argmax(T,1)) #tf.argmax(A3, 1) 1은 행

# batch_size X 10의 True, False를 1 또는 0으로 변환(tf.cast)
accuracy = tf.reduce_mean(tf.cast(predicted_val, dtype = tf.float32))

In [15]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(epochs) :
        total_batch = int(mnist.train.num_examples/batch_size) # 55000/100
        for step in range(total_batch):
            batch_x_data, batch_t_data = mnist.train.next_batch(batch_size)
            loss_val, _ = sess.run([loss, train], feed_dict = {X: batch_x_data, T : batch_t_data})
            if step % 100 == 0:
                print("step = ", step, ", loss_val = ", loss_val)
                
    test_x_data = mnist.test.images # 10000x784
    test_t_data = mnist.test.labels # 10000x10
    
    accuracy_val = sess.run(accuracy, feed_dict={X : test_x_data, T : test_t_data})
    print("\nAccuracy = ", accuracy_val)
    # MNIST와 같은 이미지데이터에 대한 인식정확도를 99%이상으로 높이기 위해서는 신경망 아키텍처를 CNN으로 전환하는 것이 필요

step =  0 , loss_val =  85.47342
step =  100 , loss_val =  4.66308
step =  200 , loss_val =  2.03977
step =  300 , loss_val =  2.923041
step =  400 , loss_val =  1.6499202
step =  500 , loss_val =  1.7070236
step =  0 , loss_val =  0.7944725
step =  100 , loss_val =  0.8444023
step =  200 , loss_val =  1.40542
step =  300 , loss_val =  1.2733067
step =  400 , loss_val =  0.81509805
step =  500 , loss_val =  0.76210624
step =  0 , loss_val =  1.7060696
step =  100 , loss_val =  0.22344132
step =  200 , loss_val =  0.974605
step =  300 , loss_val =  0.90305984
step =  400 , loss_val =  0.42978066
step =  500 , loss_val =  1.3534541
step =  0 , loss_val =  0.32209054
step =  100 , loss_val =  0.8654674
step =  200 , loss_val =  0.72328436
step =  300 , loss_val =  0.37409386
step =  400 , loss_val =  0.7021297
step =  500 , loss_val =  0.6836505
step =  0 , loss_val =  0.3708986
step =  100 , loss_val =  0.68478477
step =  200 , loss_val =  0.2962578
step =  300 , loss_val =  0.5394558
st

step =  400 , loss_val =  0.19914627
step =  500 , loss_val =  0.5116045
step =  0 , loss_val =  0.14650555
step =  100 , loss_val =  0.1750741
step =  200 , loss_val =  0.15245241
step =  300 , loss_val =  0.3672428
step =  400 , loss_val =  0.11178449
step =  500 , loss_val =  0.1691236
step =  0 , loss_val =  0.23289658
step =  100 , loss_val =  0.12346192
step =  200 , loss_val =  0.08923296
step =  300 , loss_val =  0.30790415
step =  400 , loss_val =  0.25293347
step =  500 , loss_val =  0.09012916
step =  0 , loss_val =  0.21697105
step =  100 , loss_val =  0.28342074
step =  200 , loss_val =  0.07491186
step =  300 , loss_val =  0.23760095
step =  400 , loss_val =  0.25587463
step =  500 , loss_val =  0.15526254
step =  0 , loss_val =  0.16169089
step =  100 , loss_val =  0.039699048
step =  200 , loss_val =  0.06306884
step =  300 , loss_val =  0.06981277
step =  400 , loss_val =  0.0487541
step =  500 , loss_val =  0.055874325
step =  0 , loss_val =  0.0996742
step =  100 , l

step =  0 , loss_val =  0.023376765
step =  100 , loss_val =  0.119735755
step =  200 , loss_val =  0.15679759
step =  300 , loss_val =  0.07421705
step =  400 , loss_val =  0.06728418
step =  500 , loss_val =  0.09589526
step =  0 , loss_val =  0.091936156
step =  100 , loss_val =  0.063591406
step =  200 , loss_val =  0.18193986
step =  300 , loss_val =  0.137066
step =  400 , loss_val =  0.04279043
step =  500 , loss_val =  0.032594875
step =  0 , loss_val =  0.084474914
step =  100 , loss_val =  0.09915737
step =  200 , loss_val =  0.12940568
step =  300 , loss_val =  0.06258085
step =  400 , loss_val =  0.021686118
step =  500 , loss_val =  0.26601666
step =  0 , loss_val =  0.0869393
step =  100 , loss_val =  0.16252272
step =  200 , loss_val =  0.07529534
step =  300 , loss_val =  0.08259848
step =  400 , loss_val =  0.09263971
step =  500 , loss_val =  0.07144951
step =  0 , loss_val =  0.10289358
step =  100 , loss_val =  0.12493889
step =  200 , loss_val =  0.07496108
step = 